# Evakuatsiooni teekondade mudel

## Pythoni lisade import

In [2]:
# Need koodiread on minu jaoks olnud alati vajalikud, kui kasutada ruumiandmeid pythonis 
import os
os.environ['PROJ_DATA'] = r'C:\Users\Dan\micromamba\envs\geopython2025\Library\share\proj'
os.environ['PROJ_LIB'] = r'C:\Users\\Dan\micromamba\envs\geopython\Library\share\proj'

In [3]:
from pathlib import Path
import numpy as np
from scipy.spatial import cKDTree
from skimage.graph import MCP_Geometric
from shapely.geometry import Point, LineString, box
from shapely.ops import substring
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
from rasterio.windows import from_bounds
from rasterio.features import rasterize
from rasterio.transform import xy
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm, LinearSegmentedColormap, Normalize
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from tqdm import tqdm
tqdm.pandas()

## Vektormudel

### Algandmed ja seadistus

In [18]:
output_dir = Path(r"C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output")# Defineerin kuhu andmed salvestatakse
input_dir = Path(r"C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Water_depth_DEM")# Defineerin kust võetakse sügavusandmed
# Kausta nimede kirjeldamine. Vajalik kaartide salvestamiseks. Esimese stsenaariumi kaardid salvestatakse kausta S1 jne.
folder_map = {
    'scenario1': 'S1',
    'scenario2': 'S2',
    'scenario3': 'S3'}
# Defineerin kaardiakna asukoha/suuruse
bbox = (525250.0, 6468167.0, 534250.0, 6474167.0) 
# Defineerin punktide nimed ja koordinaadid
points = {
    'Tervise Paradiis': (529890.38, 6470429.73),
    'Hedon Hotel': (529055.89, 6470713.17),
    'Riia Tammsaare rist': (531076.80, 6470926.86),
    'Paul Kerese mälestusmärk': (529626.52, 6471593.31)} 
# Defineerin algus ja lõpp punktide kombinatasioonid. Ehk millisest punktist marsruut algab ja millises lõppeb
route_pairs = [
    ('Tervise Paradiis', 'Riia Tammsaare rist'),
    ('Tervise Paradiis', 'Paul Kerese mälestusmärk'),
    ('Hedon Hotel', 'Riia Tammsaare rist'),
    ('Hedon Hotel', 'Paul Kerese mälestusmärk')]
#Defineerin üleujutus stsenaariumite nimed
scenarios = ['scenario1', 'scenario2', 'scenario3']
scenario_names = {
    'scenario1': '1.5m Üleujutus',
    'scenario2': '2.95m Üleujutus',
    'scenario3': '3.9m Üleujutus'}

# Defineerin kui ja kuhu kaardiaken on "zoomitud". Defineeritakse sisestatud punktide koordinaatide järgi. Lisatakse 500 ühikuline buffer 
all_x = [p[0] for p in points.values()]
all_y = [p[1] for p in points.values()]
zoom_buffer = 500 # puhver
zoom_bbox = (
    min(all_x) - zoom_buffer,
    max(all_x) + zoom_buffer,
    min(all_y) - zoom_buffer,
    max(all_y) + zoom_buffer)

### Kulufunktsioon / Vee aeglustava jõu funktsioon

In [19]:
def get_walking_cost(depth): # Defineerin fuktsiooni, mis annab igale läbitud meetrile hinnangu/hinna/kulu.
    base_cost = 0.7 # Hind/kulu/aeglustav jõud ilma takistusteta
    
    if depth < 0.1:      # Kuiv / Lombid
        return base_cost * 1.0 # Hind/aeglustav jõud kui lombid
    elif depth < 0.3:    # umbes pahkluuni
        return base_cost * 2.0 # Hind/aeglustav jõud  kui vesi on pahkluuni 
    elif depth < 0.6:    # Umbes põlvedeni
        return base_cost * 8.0 # Hind/aeglustav jõud  kui vesi on põlvedeni
    elif depth < 1.0:    # Umbes puusadeni
        return base_cost * 500.0 # Hind/aeglustav jõud  kui vesi on puusadeni
    else:                # Umbes rinnuni
        return base_cost * 100000.0 # Hind kui vesi on rinnuni

In [20]:
# Hinnad määrati eksponetnsiaalselt kasvavatena, kuna see aitab vältida olukorda, kus algorütm eelistab lühemat teed läbi sügavama vee kui valikus on pikem tee läbi madalama vee. 

### Teelõikudele sügavuste korjamise fuktsioon

In [21]:
# Funktsioon mis arvutab veesügavuse teelõigule
def get_safe_depth_on_edge(geom, raster_data, transform): 
    length = geom.length # Võtab tänava pikkuse
    if length == 0: return 0, 0, 0 # Kui tänaval pole pikkust annab väärtuse null
        
    # Teeb tänava joonele meetriste vahedega punktid. Neid kasutatakse sügavuse "mõõtmiseks"
    num_points = max(2, int(np.ceil(length))) 
    points = [geom.interpolate(d) for d in np.linspace(0, length, num_points)]
    # Siin hoitakse kogutud andmeid
    total_cost = 0
    max_edge_depth = 0
    accumulated_depths = []
    # Vektorist raster koorinaatsüsteemi üleviimise ettevalmistus
    inv_transform = ~transform
    rows, cols = raster_data.shape
    
    for pt in points: # iga punktiga mis teelõigule loodi tehakse järgnevad operatsioonid.
        col, row = inv_transform * (pt.x, pt.y) #Meetriste vahedega punktide koordinaadid muudetakse piksel/raster koordinaatideks. Kood saab teada, kus punkt rastril asub. Millisel real millises tulbas
        r, c = int(row), int(col)
        
        # iga "mõõtepiksli" ümber oleva akna defineerimine. Aken on 3x3 pikslit ümber "mõõtepiksli"
        r_min, r_max = max(0, r-1), min(rows, r+2)
        c_min, c_max = max(0, c-1), min(cols, c+2)
        # Leitakse 3x3 aknas sügavaim punkt 
        if r_min < r_max and c_min < c_max:
            window = raster_data[r_min:r_max, c_min:c_max]
            local_risk = np.max(window)
        else:
            local_risk = 0 # kui aken töötab rastri äärel, kus ei pruugi kõiki väärtuseid olla määratakse sügavuseks 0. Oluline errorite vältimiseks aga üldiselt ei tohiks see kasutusse minnagi.

        # Salvestab kõige sügavama sügavuse teelõigul
        max_edge_depth = max(max_edge_depth, local_risk)
        # kogub kõik sügavused teelõigul
        accumulated_depths.append(local_risk)

        # Arvutab "hinna" selle meetri läbimiseks
        # Selleks kasutatakse eelnevalt defineeritud funktsiooni "get_walking_cost" 
        step_cost = get_walking_cost(local_risk) # muudab mõõdetud sügavuse "hinnaks"
        step_dist = length / (num_points - 1) if num_points > 1 else length # mõõdab kauguse punktide vahel
        total_cost += step_cost * step_dist # lisab/liidab meetri läbimise "hinna" kogu "hinnale" juurde. Seda tehakse iga punkti koha, seeläbi saades kogu teelõigu "hinna" 

    avg_depth = np.mean(accumulated_depths) if accumulated_depths else 0 # lõpuks arvutab keskmise sügavuse kogu teelõigule.
    return total_cost, max_edge_depth, avg_depth

### Alguspunkti teedevõrguga sidumise fuktsioonid

In [22]:
# Funktsioon algus ja lõpp punktide teedevõrguga sidumiseks. Oluline kuna algus ja lõpp punktid ei ole teede võrguga "snäpitud"
def split_edge_at_point(G, u, v, key, point, point_name):
    edge_data = G[u][v][key] # Loeb sisse teedevõrgu andmed
    line = edge_data.get('geometry', LineString([
        (G.nodes[u]['x'], G.nodes[u]['y']), 
        (G.nodes[v]['x'], G.nodes[v]['y'])
    ])) # defineerib milliseid andmeid kasutada
    
    # Leiab algus/lõpp-punktile lähima punkti lähimal teelõigul
    proj_dist = line.project(point) 
    proj_point = line.interpolate(proj_dist) 
    # Poolitab tee selles lähimas punktis 
    geom_u_new = substring(line, 0, proj_dist)
    geom_new_v = substring(line, proj_dist, line.length)
    # Lisab poolitatud kohta tee "sõlmpunkti" selleks, et teekond saaks sealt alata
    new_node_id = f"split_{point_name}"
    G.add_node(new_node_id, x=proj_point.x, y=proj_point.y, geometry=Point(proj_point))
    
    base_attrs = {k:v for k,v in edge_data.items() if k not in ['geometry', 'length', 'weight']}
    # Lisab muudetud andmed tagasi võrgustikku
    G.add_edge(u, new_node_id, **base_attrs, geometry=geom_u_new, length=geom_u_new.length)
    G.add_edge(new_node_id, v, **base_attrs, geometry=geom_new_v, length=geom_new_v.length)
    G.remove_edge(u, v, key)
    
    return new_node_id
# Funktsioon mis kasutab eelmist funktsiooni koos defineeritud algus- ja lõpp-punktidega ja teedevõrguga, et muudatus ära teha.
def prepare_graph_for_route(G_input, start_pt, end_pt):
    G_temp = G_input.copy()
    u1, v1, k1 = ox.nearest_edges(G_temp, start_pt[0], start_pt[1])
    u2, v2, k2 = ox.nearest_edges(G_temp, end_pt[0], end_pt[1])
    start_node = split_edge_at_point(G_temp, u1, v1, k1, Point(start_pt), "START")
    end_node = split_edge_at_point(G_temp, u2, v2, k2, Point(end_pt), "END")
    return G_temp, start_node, end_node

### Teedevõrgu ja sügavus andmete sisselaadimine

In [23]:
# siin hoitakse raster andmeid töötluseks
depth_rasters = {} # Üleujutus stsenaariumite sügavusandmed
raster_transforms = {} # "mõõtepunktide" andmed

# Iga üleujutus stsenaariumi sisse lugemine.
for scenario in scenarios:
    depth_path = input_dir / f"water_depth_{scenario}.tif" # Defineerin sisseloetavad andmed
    if depth_path.exists(): # Kui andmed on olemas
        with rasterio.open(depth_path) as src: # Loetakse andmed sisse nende parameetritega
            window = from_bounds(*bbox, src.transform)
            data = src.read(1, window=window)
            data = np.nan_to_num(data, nan=0.0)
            data[data < 0] = 0
            depth_rasters[scenario] = data
            raster_transforms[scenario] = src.window_transform(window)
# OSM tänavavõrgustiku andmete allalaadimine
place_name = "Pärnu, Estonia" # Defineerin piirkonna
G_raw = ox.graph_from_place(place_name, network_type='walk') # tänava võrgustiku liigi
G_base = ox.project_graph(G_raw, to_crs='EPSG:3301') # Viin andmed Eesti koordinaatsüsteemi

### Töötlus

In [24]:
for scenario in scenarios: # tsükliline käsklus iga tstenaariumi kohta
    if scenario not in depth_rasters: continue
    print(f"\nTöötleb: {scenario_names[scenario]}") 

    # 1. Loo kaust stsenaariumi jaoks (nt .../Vektor/S1)
    subfolder_name = folder_map.get(scenario, scenario) # mapib scenario1 -> S1
    save_dir = output_dir / subfolder_name
    save_dir.mkdir(parents=True, exist_ok=True) # Loob kausta kui seda pole

    curr_raster = depth_rasters[scenario] 
    curr_transform = raster_transforms[scenario] 
    G_scenario = G_base.copy() 
    
    print("  Arvutan teelõikude kaalusid")
    bg_lines = [] 

    for u, v, k, data in G_scenario.edges(keys=True, data=True):
        if 'geometry' not in data:
            data['geometry'] = LineString([
                (G_scenario.nodes[u]['x'], G_scenario.nodes[u]['y']), 
                (G_scenario.nodes[v]['x'], G_scenario.nodes[v]['y'])
            ]) 
        
        bg_lines.append(data['geometry'])
        c, m, a = get_safe_depth_on_edge(data['geometry'], curr_raster, curr_transform)
        data.update({'weight': c, 'max_depth': m})

    print("  Genereerib kaarte")
    
    # KÄIME LÄBI IGA MARSRUUDI ERALDI
    for s_name, e_name in route_pairs:
        
        # Loome uue "figuuri" iga kaardi jaoks eraldi
        fig, ax = plt.subplots(figsize=(12, 10))
        
        s_coords, e_coords = points[s_name], points[e_name]
        
        G_route, s_node, e_node = prepare_graph_for_route(G_scenario, s_coords, e_coords)
        
        # Uute "splititud" lõikude sügavused
        for node in [s_node, e_node]:
            incident_edges = list(G_route.out_edges(node, keys=True, data=True)) + \
                             list(G_route.in_edges(node, keys=True, data=True))
            for u, v, k, d in incident_edges:
                if 'weight' not in d:
                    c, m, a = get_safe_depth_on_edge(d['geometry'], curr_raster, curr_transform)
                    d.update({'weight': c, 'max_depth': m})

        try:
            path = nx.shortest_path(G_route, s_node, e_node, weight='weight')
            
            path_geoms, path_depths = [], []
            max_d_route = 0 
            
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1] 
                edges = G_route[u][v] 
                best_key = min(edges, key=lambda k: edges[k].get('weight', float('inf')))
                edge = edges[best_key]
                
                max_d_route = max(max_d_route, edge.get('max_depth', 0))

                g = edge['geometry']
                if g.length > 0:
                    pts = [g.interpolate(d) for d in np.linspace(0, g.length, max(2, int(g.length)))]
                    for j in range(len(pts)-1):
                        micro_seg = LineString([pts[j], pts[j+1]])
                        path_geoms.append(micro_seg)
                        mid = micro_seg.interpolate(micro_seg.length/2)
                        r, c_idx = ~curr_transform * (mid.x, mid.y)
                        try: val = curr_raster[int(c_idx), int(r)]
                        except: val = 0
                        path_depths.append(val)

            #    KIHTIDE LISAMINE 
            
            # tänavate leppemärk
            lc_bg = LineCollection([list(g.coords) for g in bg_lines], 
                                   colors='#999999', linewidths=0.5, alpha=0.3, zorder=1)
            ax.add_collection(lc_bg)
            
            # Veesügavuse raster ja VÄRVISKAALA 
            masked = np.ma.masked_where(curr_raster <= 0.05, curr_raster)
            
            # Salvestame pildi objekti muutujasse 'img', et saaksime sellest teha scale bar'i
            img = ax.imshow(masked, cmap='magma', vmin=0, vmax=2.0, alpha=0.6, 
                      extent=[bbox[0], bbox[2], bbox[1], bbox[3]], zorder=2)
            
            # Lisame sügavuse skaala
            cbar = plt.colorbar(img, ax=ax, fraction=0.035, pad=0.04)
            cbar.set_label('Veesügavus (m)', rotation=270, labelpad=15, fontweight='bold')
            
            # Marsruudi leppemörk
            cmap_route = LinearSegmentedColormap.from_list("safety", 
                   [(0.0, "green"), (0.1, "#ADFF2F"), (0.3, "orange"), (0.6, "red"), (1.0, "black")])
            
            lc = LineCollection([list(g.coords) for g in path_geoms], 
                                array=np.array(path_depths), cmap=cmap_route, norm=plt.Normalize(0, 1.2), 
                                linewidth=4, zorder=5, capstyle='round')
            ax.add_collection(lc)
            
            # Punktid ja kaardikirjad
            ax.plot(*s_coords, marker='D', color='white', markeredgecolor='black', markersize=10, zorder=6)
            ax.text(s_coords[0], s_coords[1]-100, s_name, ha='center', fontsize=10, fontweight='bold', 
                    bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.3'), zorder=7)
            
            ax.plot(*e_coords, marker='o', color='white', markeredgecolor='black', markersize=10, zorder=6)
            ax.text(e_coords[0], e_coords[1]+100, e_name, ha='center', fontsize=10, fontweight='bold',
                    bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.3'), zorder=7)
            
            # Pealkiri
            ax.set_title(f"{scenario_names[scenario]}\n{s_name} -> {e_name}\nMax sügavus teekonnal: {max_d_route:.2f}m", 
                         fontweight='bold', fontsize=14)

        except nx.NetworkXNoPath:
            ax.text(0.5, 0.5, "TEE BLOKEERITUD", transform=ax.transAxes, ha='center', 
                    color='red', fontsize=20, fontweight='bold')
            ax.set_title(f"{scenario_names[scenario]}\n{s_name} -> {e_name} (BLOKEERITUD)", fontweight='bold')

        # kaardiakna seadistamine
        ax.set_xlim(zoom_bbox[0], zoom_bbox[1])
        ax.set_ylim(zoom_bbox[2], zoom_bbox[3])
        ax.axis('off')

        # Teekonna ohutuse legend (all vasakul)
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='green', lw=4, label='Kuiv (<0.1m)'),
            Line2D([0], [0], color='#ADFF2F', lw=4, label='Kerge (0.1-0.3m)'),
            Line2D([0], [0], color='orange', lw=4, label='Keskmine (0.3-0.6m)'),
            Line2D([0], [0], color='red', lw=4, label='Raske (0.6-1.0m)'),
            Line2D([0], [0], color='black', lw=4, label='Eluohtlik (>1.0m)')]
        
        ax.legend(handles=legend_elements, loc='lower right', fontsize=10, frameon=True, framealpha=0.9)

        # Salvestamine
        # Teeme failinime "puhtaks" (asendame tühikud alakriipsudega)
        safe_s_name = s_name.replace(" ", "_")
        safe_e_name = e_name.replace(" ", "_")
        
        filename = f"{safe_s_name}_to_{safe_e_name}.png"
        output_file = save_dir / filename
        
        plt.tight_layout()
        plt.savefig(output_file, dpi=300)
        print(f"    Salvestatud: {output_file}")
        
        # töötluse lõpp
        plt.close(fig)
print("\nKõik vektor-stsenaariumid on töödeldud.")


Töötleb: 1.5m Üleujutus
  Arvutan teelõikude kaalusid
  Genereerib kaarte
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S1\Tervise_Paradiis_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S1\Tervise_Paradiis_to_Paul_Kerese_mälestusmärk.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S1\Hedon_Hotel_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S1\Hedon_Hotel_to_Paul_Kerese_mälestusmärk.png

Töötleb: 2.95m Üleujutus
  Arvutan teelõikude kaalusid
  Genereerib kaarte
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S2\Tervise_Paradiis_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Vector_Output\S2\Tervise_Paradiis_to_Paul_Kerese_mälestusmärk.png
    Salvestatud: C:\Users\Dan\Documents\GitHub

## Rastermudel

### Algandmed ja seadistus

In [32]:
base_dir = Path(r"C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel") 
output_dir = base_dir / "Raster_Output" # Defineerin kuhu andmed salvestatakase
input_dir = base_dir / "Water_depth_DEM" # Defineerin kust võetakse vee sügavuse DEM
ref_raster_path = base_dir / "Height_DEM" / "merged_DEM_1m.tif" # Defineerin kust andmed võetakse
# Kausta nimede kirjeldamine. Vajalik kaartide salvestamiseks. Esimese stsenaariumi kaardid salvestatakse kausta S1 jne.
folder_map = {
    'scenario1': 'S1',
    'scenario2': 'S2',
    'scenario3': 'S3'}
# Defineerin kaardiakna asukoha/suuruse
points = {
    'Tervise Paradiis': (529890.38, 6470429.73),
    'Hedon Hotel': (529055.89, 6470713.17),
    'Riia Tammsaare rist': (531076.80, 6470926.86),
    'Paul Kerese mälestusmärk': (529626.52, 6471593.31)}
# Defineerin algus ja lõpp punktide kombinatasioonid. Ehk millisest punktist marsruut algab ja millises lõppeb
route_pairs = [
    ('Tervise Paradiis', 'Riia Tammsaare rist'),
    ('Tervise Paradiis', 'Paul Kerese mälestusmärk'),
    ('Hedon Hotel', 'Riia Tammsaare rist'),
    ('Hedon Hotel', 'Paul Kerese mälestusmärk')]
#Defineerin üleujutus stsenaariumite nimed
scenarios = {
    'scenario1': '1.5m Üleujutus',
    'scenario2': '2.95m Üleujutus',
    'scenario3': '3.9m Üleujutus'}
# Defineerin kui ja kuhu kaardiaken on "zoomitud". Defineeritakse sisestatud punktide koordinaatide järgi. Lisatakse 500 ühikuline buffer 
zoom_bbox = (528500.0, 6470000.0, 531500.0, 6472000.0)
calc_buffer = 1000
process_bbox = (
    zoom_bbox[0] - calc_buffer,
    zoom_bbox[1] - calc_buffer,
    zoom_bbox[2] + calc_buffer,
    zoom_bbox[3] + calc_buffer)

### Teedevõrgu rasteriseerimise funktsioon

In [33]:
def create_walkable_mask(raster_path, bbox):    
    with rasterio.open(raster_path) as src:
        window = from_bounds(*bbox, transform=src.transform)
        win_transform = src.window_transform(window)
        height = int(window.height)
        width = int(window.width)
        shape = (height, width)
        print(f"  Raster akna suurus: {shape}")

    print("  teedevõrgu rasteriseerimine")
    
    try:
        place_name = "Pärnu, Estonia"
        G = ox.graph_from_place(place_name, network_type='walk')
        G = ox.project_graph(G, to_crs='EPSG:3301')
        
        gdf_edges = ox.graph_to_gdfs(G, nodes=False)
        # Teelõikude puhverdamine selleks et, luua raster teedevõrk
        gdf_edges['geometry'] = gdf_edges.geometry.buffer(2.5)
        
        # Andmete parajaks lõikamine selleks et vähendada arvutusmahtu
        bbox_geom = box(*bbox)
        gdf_clipped = gdf_edges.clip(bbox_geom)
        
        if gdf_clipped.empty:
            print(" teedevõrk puudu")
            osm_mask = np.zeros(shape, dtype=np.uint8)
        else:
            osm_shapes = [(geom, 1) for geom in gdf_clipped.geometry]
            osm_mask = rasterize(
                osm_shapes,
                out_shape=shape,
                transform=win_transform,
                fill=0,
                dtype=np.uint8
            )
            print("teedevõrk edukalt rasteriseeritud")
        
    except Exception as e:
        print(f"  OSM-i Error: {e}")
        osm_mask = np.zeros(shape, dtype=np.uint8)

    return osm_mask, win_transform, window

### Hindade määramise funktsioon

In [34]:
def get_friction_surface(depth_data, walkable_mask): # Funktsioon mis arvutab vee aeglustava jõu veesügavuse põhjal

    # Väljaspool teedevõrku on aeglustav jõud lõpmatu selleks, et veenduda et algorütm arvutab teekonnad ainult mööda defineeritud teedevõrku
    cost_surface = np.full(depth_data.shape, np.inf, dtype=np.float32)
    
    # Hind/kulu/aeglustav jõud ilma takistusteta
    base_cost = 0.7 
    
    water_costs = np.full_like(depth_data, base_cost)
    
    # umbes pahkluuni
    mask_ankle = (depth_data >= 0.1) & (depth_data < 0.3)
    water_costs[mask_ankle] = base_cost * 2.0 # Hind/aeglustav jõud  kui vesi on pahkluuni
    
    # Umbes põlvedeni
    mask_knee = (depth_data >= 0.3) & (depth_data < 0.6)
    water_costs[mask_knee] = base_cost * 8.0 # Hind/aeglustav jõud  kui vesi on põlvedeni 
    
    # Umbes puusadeni
    mask_waist = (depth_data >= 0.6) & (depth_data < 1.0)
    water_costs[mask_waist] = base_cost * 500.0 # Hind/aeglustav jõud  kui vesi on puusadeni
    
    # Umbes rinnuni
    mask_chest = (depth_data >= 1.0)
    water_costs[mask_chest] = base_cost * 100000.0 # Hind kui vesi on rinnuni
    
    # Aeglustava jõu muutujad määratakse ainult pikslitele mis on teedevõrgu osa.
    mask_bool = walkable_mask.astype(bool)
    cost_surface[mask_bool] = water_costs[mask_bool]
    
    return cost_surface

### Algus- ja lõpp-punkti teedevõrgustikuga sidumise funktsioon

In [35]:
def snap_to_mask(coord, valid_tree, valid_rc, transform): # Funktsioon mis leiab lähima teedevõrgu piksli määratud punktidele
    x, y = coord
    c_float, r_float = ~transform * (x, y) 
    dist, idx = valid_tree.query([r_float, c_float])
    nearest_rc = valid_rc[idx]
    return tuple(nearest_rc)

### Töötlus

In [36]:
# Teedevõrgu "maski" genereerimine 
walkable_mask, win_transform, window = create_walkable_mask(ref_raster_path, process_bbox)

# Ruumilise indeksi loomine selleks et leida lähim teedevõrgu piksel alguspunktile.
print("Ruumilise indeksi loomine")
valid_pixels_rc = np.argwhere(walkable_mask)

if len(valid_pixels_rc) == 0:
    raise ValueError("Alguspunkt ei leia teedevõrgu piksleid, kontrolli OSM-i teedevõrku ja koordinaatsüsteeme")

valid_tree = cKDTree(valid_pixels_rc)

  Raster akna suurus: (4000, 5000)
  teedevõrgu rasteriseerimine
teedevõrk edukalt rasteriseeritud
Ruumilise indeksi loomine


In [37]:
for scenario_key, scenario_label in scenarios.items(): # tsükliline käsklus iga tstenaariumi kohta
    print(f"\ntöötleb: {scenario_label}...")
    
    # stsenaariumi kaustade loomine
    subfolder_name = folder_map.get(scenario_key, scenario_key)
    save_dir = output_dir / subfolder_name
    save_dir.mkdir(parents=True, exist_ok=True)

    depth_file = input_dir / f"water_depth_{scenario_key}.tif"
    if not depth_file.exists(): # Teavitus juhul kui veesügavus raster fail puudu on.
        print(f"jätab vahele {depth_file} (faili ei leitud)")
        continue
        
    # Veesügavus rastri sisselugemine
    with rasterio.open(depth_file) as src:
        depth_data = src.read(1, window=window)
        depth_data = np.nan_to_num(depth_data, nan=0.0)
        depth_data[depth_data < 0] = 0

    print(" aeglustava jõu arvutused igale teedevõrgu pikslile")
    friction = get_friction_surface(depth_data, walkable_mask)
    mcp = MCP_Geometric(friction)

    # Kaardiakna suuruse leidmine iga kaardi jaoks
    h, w = depth_data.shape
    left = win_transform[2]
    top = win_transform[5]
    right = left + (w * win_transform[0])
    bottom = top + (h * win_transform[4]) 
    extent = [left, right, bottom, top]

    print("Kaartide joonistamine")
    
    # IGA MARSRUUT ERALDI
    for start_name, end_name in route_pairs:
        
        # Loome uue kaardi/graafiku
        fig, ax = plt.subplots(figsize=(12, 10))

        # Alguspunkt ja lõpp-punkti ühedamine teedevõrguga
        start_rc = snap_to_mask(points[start_name], valid_tree, valid_pixels_rc, win_transform)
        end_rc = snap_to_mask(points[end_name], valid_tree, valid_pixels_rc, win_transform)
        
        path_found = False
        route_x, route_y, route_depths = [], [], []
        max_depth = 0
        
        try:
            # Leiab teekonna kogu hinna/ aeglustava jõu
            cumulative_costs, traceback = mcp.find_costs(starts=[start_rc], ends=[end_rc])
            
            if np.isinf(cumulative_costs[end_rc]):
                raise ValueError("Unreachable")
                
            route_rc = mcp.traceback(end_rc)
            
            # teekonna pikslite sügavuse kogumine
            for r, c in route_rc:
                x_geo, y_geo = xy(win_transform, r, c, offset='center')
                route_x.append(x_geo)
                route_y.append(y_geo)
                route_depths.append(depth_data[r, c])
            
            if len(route_x) > 1:
                path_found = True
                max_depth = np.max(route_depths)
                
        except Exception:
            path_found = False

        #KIHTIDE LISAMINE
        
        #  teedevõrk
        road_display = np.ma.masked_where(walkable_mask == 0, walkable_mask)
        ax.imshow(road_display, cmap='Greys', vmin=0, vmax=1.2, alpha=0.3, extent=extent, zorder=1)
        
        # Veesügavuse raster ja VÄRVISKAALA 
        water_display = np.ma.masked_where(depth_data <= 0.05, depth_data)
        img = ax.imshow(water_display, cmap='magma', vmin=0, vmax=2.0, alpha=0.6, extent=extent, zorder=2)
        
        # sügavuse skaala
        cbar = plt.colorbar(img, ax=ax, fraction=0.035, pad=0.04)
        cbar.set_label('Veesügavus (m)', rotation=270, labelpad=15, fontweight='bold')
        
        # Marsruut
        if path_found:
            points_arr = np.array([route_x, route_y]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points_arr[:-1], points_arr[1:]], axis=1)
            
            # Samad värvid mis vektoril
            cmap_route = LinearSegmentedColormap.from_list("safety", 
                   [(0.0, "green"), (0.1, "#ADFF2F"), (0.3, "orange"), (0.6, "red"), (1.0, "black")])
            
            lc_fg = LineCollection(segments, cmap=cmap_route, norm=Normalize(0, 1.2), 
                                   linewidth=4, zorder=5, capstyle='round')
            lc_fg.set_array(np.array(route_depths))
            ax.add_collection(lc_fg)
            
            ax.set_title(f"{scenario_label}\n{start_name} -> {end_name}\nMax sügavus teekonnal: {max_depth:.2f}m", 
                         fontweight='bold', fontsize=14)
        else:
            ax.text(0.5, 0.5, "TEE BLOKEERITUD", transform=ax.transAxes, ha='center', 
                   color='red', fontsize=20, fontweight='bold', bbox=dict(facecolor='white', alpha=0.8))
            ax.set_title(f"{scenario_label}\n{start_name} -> {end_name} (BLOKEERITUD)", fontweight='bold')

        # punktleppemärgid
        ax.plot(*points[start_name], marker='D', color='white', mec='black', ms=10, zorder=6)
        ax.text(points[start_name][0], points[start_name][1]-100, start_name, ha='center', 
                fontsize=10, fontweight='bold', bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.3'), zorder=7)

        ax.plot(*points[end_name], marker='o', color='white', mec='black', ms=10, zorder=6)
        ax.text(points[end_name][0], points[end_name][1]+100, end_name, ha='center', 
                fontsize=10, fontweight='bold', bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.3'), zorder=7)
        
        # Seadistame vaate
        ax.set_xlim(zoom_bbox[0], zoom_bbox[2])
        ax.set_ylim(zoom_bbox[1], zoom_bbox[3])
        ax.axis('off')

        # Legend
        legend_elements = [
            Line2D([0], [0], color='green', lw=4, label='Kuiv (<0.1m)'),
            Line2D([0], [0], color='#ADFF2F', lw=4, label='Kerge (0.1-0.3m)'),
            Line2D([0], [0], color='orange', lw=4, label='Keskmine (0.3-0.6m)'),
            Line2D([0], [0], color='red', lw=4, label='Raske (0.6-1.0m)'),
            Line2D([0], [0], color='black', lw=4, label='Eluohtlik (>1.0m)')
        ]
        ax.legend(handles=legend_elements, loc='lower right', fontsize=10, frameon=True, framealpha=0.9)
        
        # Salvestamine
        safe_s_name = start_name.replace(" ", "_")
        safe_e_name = end_name.replace(" ", "_")
        filename = f"{safe_s_name}_to_{safe_e_name}.png"
        out_img = save_dir / filename
        
        plt.tight_layout()
        plt.savefig(out_img, dpi=300)
        print(f"    Salvestatud: {out_img}")
        
        plt.close(fig)

print("\nKõik raster-stsenaariumid on töödeldud.")


töötleb: 1.5m Üleujutus...
 aeglustava jõu arvutused igale teedevõrgu pikslile
Kaartide joonistamine
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S1\Tervise_Paradiis_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S1\Tervise_Paradiis_to_Paul_Kerese_mälestusmärk.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S1\Hedon_Hotel_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S1\Hedon_Hotel_to_Paul_Kerese_mälestusmärk.png

töötleb: 2.95m Üleujutus...
 aeglustava jõu arvutused igale teedevõrgu pikslile
Kaartide joonistamine
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S2\Tervise_Paradiis_to_Riia_Tammsaare_rist.png
    Salvestatud: C:\Users\Dan\Documents\GitHub\Evakuatsiooniteede_mudel\Raster_Output\S2\Tervise_Paradiis_to_Paul_Kerese_mälestusm